<a href="https://colab.research.google.com/github/nurpurwanto/EDM-FINPRO-ECOMMERCE-SHIPPING/blob/main/STAGE_2_DATA_PREPROCESSING_TOKOTRONIK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**An international e-commerce company based wants to discover key insights from their customer database. They want to use some of the most advanced machine learning techniques to study their customers. The company sells electronic products.**

**Data Description**

**ID**: ID Number of Customers.

**Warehouse block**: The Company have big Warehouse which is divided in to block such as A,B,C,D,E.

**Mode of shipment**:The Company Ships the products in multiple way such as Ship, Flight and Road.

**Customer care calls**: The number of calls made from enquiry for enquiry of the shipment.

**Customer rating**: The company has rated from every customer. 1 is the lowest (Worst), 5 is the highest (Best).

**Cost of the product**: Cost of the Product in US Dollars.

**Prior purchases**: The Number of Prior Purchase.

**Product importance**: The company has categorized the product in the various parameter such as low, medium, high.

**Gender**: Male and Female.

**Discount offered**: Discount offered on that specific product.

**Weight in gms**: It is the weight in grams.

**Reached on time**: It is the target variable, where 1 Indicates that the product has NOT reached on time and 0 indicates it has reached on time.


[Source Dataset](https://www.kaggle.com/datasets/prachi13/customer-analytics?select=Train.csv)

#Import Library

In [ ]:
#Lib for dataframe, agg, and data viz
from IPython import display
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 0
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set_theme(style="whitegrid")
import numpy as np
from textwrap import wrap
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

#lib for preprocessing
# Library for Scalling Data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import boxcox
import warnings
# Library for Split Data
from sklearn.model_selection import train_test_split
# Library for Z-Score
from scipy import stats

# Evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_validate

# Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

## Load Data Frame

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/FINPRO RAKAMIN 29/

In [ ]:
df = pd.read_csv('Train.csv')
df.sample(5)

In [ ]:

df=df.drop("ID", axis='columns')

df['Customer_rating']=df['Customer_rating'].astype(np.object)
df['Reached.on.Time_Y.N']=df['Reached.on.Time_Y.N'].astype(np.object)
df.info()

In [ ]:
df_numerical = df.select_dtypes(exclude='object')
df_categorical = df.select_dtypes(include='object')

In [ ]:
numerical = df_numerical.columns.values
categorical = df_categorical.columns.values

#Data Cleansing (50 poin) 


Lakukan pembersihan data, sesuai yang diajarkan di kelas, seperti:

1. Handle missing values

2. Handle duplicated data

3. Handle outliers

4. Feature transformation

5. Feature encoding

6. Handle class imbalance

Di laporan homework, tuliskan apa saja yang telah dilakukan dan metode yang digunakan.* Tetap tuliskan jika memang ada tidak yang perlu di-handle (contoh: “Tidak perlu feature encoding karena semua feature sudah numerical” atau “Outlier tidak di-handle karena akan fokus menggunakan model yang robust terhadap outlier”).





#Feature Engineering (35 poin) 

Cek feature yang ada sekarang, lalu lakukan:

A. Feature selection (membuang feature yang kurang relevan atau redundan)

B. Feature extraction (membuat feature baru dari feature yang sudah ada)

C. Tuliskan minimal 4 feature tambahan (selain yang sudah tersedia di dataset) yang mungkin akan sangat membantu membuat performansi model semakin bagus (ini hanya ide saja, untuk menguji kreativitas teman-teman, tidak perlu benar-benar dicari datanya dan tidak perlu diimplementasikan)

* Untuk 2A & 2B, tetap tuliskan jika memang tidak bisa dilakukan (contoh: “Semua feature digunakan untuk modelling (tidak ada yang dihapus), karena semua feature relevan”)


# Stage 2 Pre-processing Data

## Handle Outliers

### Z-Score Outliers

In [ ]:
df_outliers = ['Prior_purchases', 'Discount_offered']

In [ ]:
from scipy.stats import zscore

for i in df_outliers:
    print(i)
    z_scores = zscore(df[i])
    print("*"*50)
    for threshold in np.arange(1,5,0.5):
        print("Treshold: {}".format(threshold))
        print("Amount of Outliers: {}".format(len((np.where(z_scores >= threshold)[0]))))
        print('------')

### Decision

In [ ]:
print(f'Number of rows before deleting outliers: {len(df)}')

filtered_entries = np.array([True] * len(df))

for col in ['Prior_purchases','Discount_offered']:
    zscore = abs(stats.zscore(df[col]))
    filtered_entries = (zscore <= 2) & filtered_entries # keep data 
    
dfz = df[filtered_entries]

print(f'Number of row after deleting outliers: {len(dfz)}')

## Encoding

### Gender Encoding

In [ ]:
map_gender = {'M' : 0, 'F':1}
dfz['enc_gender'] = df['Gender'].map(map_gender)

### Product Importance Encoding

In [ ]:
map_Pi = {'low' : 0, 'medium':1, 'high':2}
dfz['enc_Product_importance'] = df['Product_importance'].map(map_Pi)

### Mode of Shipment Encoding

In [ ]:
ohe_ship = pd.get_dummies(df['Mode_of_Shipment'], prefix='mode')
dfz = dfz.join(ohe_ship)

### Warehouse Block Encoding

In [ ]:
ohe_ware = pd.get_dummies(df['Warehouse_block'], prefix='Warehouse')
df_final = dfz.join(ohe_ware)

In [ ]:
df_final

In [ ]:
from sklearn.preprocessing import StandardScaler as ss
df_final['std_Customer_care_calls'] = ss().fit_transform(df_final['Customer_care_calls'].values.reshape(len(df_final),1))
df_final['std_Customer_rating'] = ss().fit_transform(df_final['Customer_rating'].values.reshape(len(df_final),1))
df_final['std_Cost_of_the_Product'] = ss().fit_transform(df_final['Cost_of_the_Product'].values.reshape(len(df_final),1))
df_final['std_Prior_purchases'] = ss().fit_transform(df_final['Prior_purchases'].values.reshape(len(df_final),1))
df_final['std_Discount_offered'] = ss().fit_transform(df_final['Discount_offered'].values.reshape(len(df_final),1))
df_final['std_Weight_in_gms'] = ss().fit_transform(df_final['Weight_in_gms'].values.reshape(len(df_final),1))

In [ ]:
df_final

In [ ]:
df_end = df_final.drop(columns = ['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
                                  'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
                                  'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms', 'enc_gender'])

In [ ]:
df_end